# Process Business Data

In [1]:
from collections import Counter
import json
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

## Read Business Data

The features in `business.json` file

* `business_id`: (`str`) unique id of the business

* `name`: (`str)` the business' name

* `address`: (`str`) the full address of the business

* `city`: (`str`) the city where the business is located

* `state`: (`str`) the state where the business is located

* `postal code`: (`str`) the postal code of the business

* `latitude`: (`float`) latitude

* `longitude`: (`float`) longitude

* `stars`: (`float`) average star rating, rounded to half-stars

* `review_count`: (`str`) total number of reviews given to the business

* `is_open`: (`int`) (binary) indicates whether the business is still open

* `attributes`: (`json`) attributes of the business

* `categories`: (`list`) description of the business

* `hours`: (`json`) the working hours of the business

* The businesses in the `business.json` are located in North America, mainly in the US and Canada. The main subject of this study is the restaurants in the US. For this reason, we will focus on those in the US and discard all others. However, businesses in the US are mainly clustered around Arizona-Nevada, Ohio-Pennsylvania and North Carolina rather than scatter across the country. Accordingly, when we mention 'the business in the US', we mean the business that are located in those states. 
* The feature, `is_open`, provides information about if a business is still open or closed. In the study our sample will cover the business which are in the restaurant category and are still in the business. For this reason, we will discard all businesses that are closed.
* The feature, `attributes`, contains the attributes of a business. However, those attributes are not consistent among all businesses. In the first place, we acknowledge that there will be the attributes for the businesses that are in different categories will be, indeed, different. However, those attributes are not, even, consistent among the restaurant businesses. For this reason, we will keep the attributes which have %25 or less missing values.
* Finally, `hours` feature indicates the business hours for each business. Unfortunately, it has a great amount of missing values. For this reason, it will also be discarded from the study.

In [2]:
df_business = pd.read_json('yelp_academic_dataset_business.json', lines=True)
print('First 5 rows of the business data which provides information about', end='')
print(' {:,} businesses with {} features:'.format(df_business.shape[0],
                                                df_business.shape[1]))
display(df_business.head())

First 5 rows of the business data which provides information about 209,393 businesses with 14 features:


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


### Preprocessing

In [3]:
def clean_json(text):
    """Clears the problems with the Attributes JSON objects"""
    if not isinstance(text, str):
        text = str(text)
    text = text.replace('True', '"True"')
    text = text.replace('False', '"False"')
    text = text.replace("'", '"')
    text = text.replace('None', '"False"')
    text = text.replace('"DriveThr', '"DriveThr"')
    text = text.replace('""', '"')
    text = text.replace('u"', '')
    text = text.replace('"{', '{').replace('}"', '}')
    return text

def make_json(text):
    """Transform a string object into a JSON object"""
    return json.loads(text)

def get_list(x):
    """Returns a list of categories if applicable else returns a string."""
    try:
        return x.split(', ')
    except:
        return 'NA'

In [4]:
df_business.drop(['postal_code', 'hours'], axis=1, inplace=True)
# removes the erronous characters in the `attributes` feature and 
# converts `categories` into a list
df_business.attributes = df_business.attributes.apply(clean_json)
df_business.attributes = df_business.attributes.apply(make_json)
df_business.categories = df_business.categories.apply(get_list)

In [5]:
# drops closed businesses
n_closed = df_business[df_business.is_open == 0].shape[0]
n_open = df_business[df_business.is_open == 1].shape[0]
print('The number of businesses that are closed: {:,}'.format(n_closed))
print('The number of businesses that are still in the business: {:,}'.format(n_open))
# selects all business that are still open
df = df_business.copy()
df = df.loc[df.is_open == 1]
df.drop('is_open', axis=1, inplace=True)

The number of businesses that are closed: 40,490
The number of businesses that are still in the business: 168,903


In [6]:
# Businesses with categorical data
print('Business with category information: {:,}'.format(df[df.categories != 'NA'].shape[0]))
print('Businesses with no category information: {}'.format(
    df[df.categories == 'NA'].shape[0]))

Business with category information: 168,401
Businesses with no category information: 502


In [7]:
# drop all business without categorical data
df = df.loc[df.categories != 'NA', :]
df.reset_index(drop=True, inplace=True)
display(df.head())

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,35.462724,-80.852612,3.5,36,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","[Active Life, Gun/Rifle Ranges, Guns & Ammo, S..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,33.569404,-111.890264,5.0,4,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","[Health & Medical, Fitness & Instruction, Yoga..."
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,45.479984,-73.580070,5.0,5,False,"[Pets, Pet Services, Pet Groomers]"
3,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,33.428065,-111.726648,4.5,26,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","[Home Services, Plumbing, Electricians, Handym..."
4,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,33.350399,-111.827142,4.5,38,{'BusinessAcceptsCreditCards': 'True'},"[Auto Repair, Automotive, Oil Change Stations,..."


In [8]:
# classify businesses based on their categories
# all major categories in US
# https://www.yelp.com/developers/documentation/v3/category_list
d_categories = {
    'active': 'Active Life',
    'arts': 'Arts & Entertainment',
    'auto': 'Automotive',
    'beautysvc': 'Beauty & Spas',
    'education': 'Education',
    'eventservices': 'Event Planning & Services',
    'financialservices': 'Financial Services',
    'food': 'Food',
    'health': 'Health & Medical',
    'homeservices': 'Home Services',
    'hotelstravel': 'Hotels & Travel',
    'localflavor': 'Local Flavor',
    'localservices': 'Local Services',
    'massmedia': 'Mass Media',
    'nightlife': 'Nightlife',
    'pets': 'Pets',
    'professional': 'Professional Services',
    'publicservicesgovt': 'Public Services & Government',
    'realestate': 'Real Estate',
    'religiousorgs': 'Religious Organizations',
    'restaurants': 'Restaurants',
    'shopping': 'Shopping'
}

# generates a binary variable for each category
for category in d_categories:
    df[category] = df.categories.apply(
        lambda x: 1 if d_categories[category] in x else 0)

# drop category from the dataset
df.drop('categories', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
print('First 5 rows of the business data after adding categories as features:')
display(df.head())

First 5 rows of the business data after adding categories as features:


,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,...,localservices,massmedia,nightlife,pets,professional,publicservicesgovt,realestate,religiousorgs,restaurants,shopping
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,35.462724,-80.852612,3.5,36,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",...,0,0,0,0,0,0,0,0,0,1
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,33.569404,-111.890264,5.0,4,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...",...,0,0,0,0,0,0,0,0,0,0
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,45.479984,-73.580070,5.0,5,False,...,0,0,0,1,0,0,0,0,0,0
3,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,33.428065,-111.726648,4.5,26,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",...,0,0,0,0,0,0,0,0,0,0
4,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,33.350399,-111.827142,4.5,38,{'BusinessAcceptsCreditCards': 'True'},...,0,0,0,0,0,0,0,0,0,0


In [9]:
# select all businesses in the US
state_codes = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID',
    'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
    'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
    'WI', 'WY'
]

state_names = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
    'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
    'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
    'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
    'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
    'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

d_states = {code: name for code, name in zip(state_codes, state_names)}
# generate a binary variable indicates if a business is located in the US
df['in_US'] = df.state.apply(lambda x: 1 if x in d_states else 0)

In [10]:
print('Number of businesses in the US: {:,}'.format(df['in_US'].sum()))
print('Number of businesses outside of the US: {:,}'.format(df.shape[0] -
                                                            df['in_US'].sum()))

Number of businesses in the US: 124,836
Number of businesses outside of the US: 43,565


In [11]:
# selects all businesses in the US
df = df.drop('in_US', axis=1).loc[df.in_US == 1, :]
df.reset_index(drop=True, inplace=True)
print('Example of businesses in the US:')
display(df.head())

Example of businesses in the US:


,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,...,localservices,massmedia,nightlife,pets,professional,publicservicesgovt,realestate,religiousorgs,restaurants,shopping
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,35.462724,-80.852612,3.5,36,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",...,0,0,0,0,0,0,0,0,0,1
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,33.569404,-111.890264,5.0,4,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...",...,0,0,0,0,0,0,0,0,0,0
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,33.428065,-111.726648,4.5,26,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",...,0,0,0,0,0,0,0,0,0,0
3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,33.350399,-111.827142,4.5,38,{'BusinessAcceptsCreditCards': 'True'},...,0,0,0,0,0,0,0,0,0,0
4,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,"6870 S Rainbow Blvd, Ste 117",Las Vegas,NV,36.063977,-115.241463,3.5,81,"{'BusinessParking': {'garage': 'False', 'stree...",...,1,0,0,0,0,0,0,0,0,0


In [12]:
df1 = df.copy()
df1.drop(['address', 'city', 'latitude', 'longitude', 'attributes'], axis=1, inplace=True)
df1.to_csv('business_with_categories.csv', index=False)

In [13]:
df1 = df1.loc[:, ['business_id', 'restaurants', 'shopping']]
df1.head()

,business_id,restaurants,shopping
0,f9NumwFMBDn751xgFiRbNA,0,1
1,Yzvjg0SayhoZgCljUJRF9Q,0,0
2,51M2Kk903DFYI6gnB5I6SQ,0,0
3,cKyLV5oWZJ2NudWgqs8VZw,0,0
4,oiAlXZPIFm2nBCt0DHLu_Q,0,0


In [14]:
df1[df1.restaurants==1].business_id.to_frame().to_csv('restaurants_ids.csv', index=False)
df1[df1.shopping==1].business_id.to_frame().to_csv('shopping_ids.csv', index=False)

In [15]:
# select all restaurant businesses
df = df.loc[df.restaurants == 1, :]
df.reset_index(drop=True, inplace=True)
df = df.iloc[:, :10]
df.drop(['address', 'latitude', 'longitude'], axis=1, inplace=True)
print('Total number of businesses (sample size): {:,}'.format(df.shape[0]))
print('Example of businesses in the restaurant category in the US:')
display(df.head())

Total number of businesses (sample size): 25,827
Example of businesses in the restaurant category in the US:


,business_id,name,city,state,stars,review_count,attributes
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,"{'RestaurantsAttire': 'casual', 'RestaurantsTa..."
1,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,Phoenix,AZ,4.0,10,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ..."
2,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,Cleveland,OH,4.5,23,"{'BusinessParking': 'False', 'Ambience': {'tou..."
3,tLpkSwdtqqoXwU0JAGnApw,Wendy's,Cleveland,OH,3.5,7,"{'RestaurantsDelivery': 'False', 'BusinessAcce..."
4,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,Cleveland,OH,3.0,4,"{'Alcohol': 'none', 'GoodForKids': 'True', 'Re..."


### Explore Business Attributes

* The business attributes do not consistent among businesses. Some businesses have more attributes with great detail; however some only have superficial attributes. Below, two examples are provided for the business attributes.

In [16]:
df['has_attributes'] = df.attributes.apply(lambda x: 1 if isinstance(x, dict) else 0)
mask = df.has_attributes == 1
df_attributes = pd.json_normalize(df.loc[mask, 'attributes'].values)
df_attributes['business_id'] = df[mask].business_id.values
df = df.merge(df_attributes, on='business_id', how='left')
display(df_attributes.head())

,RestaurantsAttire,RestaurantsTakeOut,BusinessAcceptsCreditCards,NoiseLevel,GoodForKids,RestaurantsReservations,RestaurantsGoodForGroups,BusinessParking,RestaurantsPriceRange2,HasTV,...,HairSpecializesIn.straightperms,HairSpecializesIn.coloring,HairSpecializesIn.extensions,HairSpecializesIn.africanamerican,HairSpecializesIn.curly,HairSpecializesIn.kids,HairSpecializesIn.perms,HairSpecializesIn.asian,DietaryRestrictions,business_id
0,casual,True,True,quiet,False,False,False,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pQeaRpvuhoEqudo3uymHIQ
1,casual,True,True,NaN,True,False,True,NaN,1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vjTVxnsQEZ34XjYNS-XUpA
2,NaN,True,True,NaN,NaN,NaN,NaN,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98hyK2QEUeI8v2y0AghfZA
3,casual,True,True,average,True,False,True,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tLpkSwdtqqoXwU0JAGnApw
4,formal,False,False,very_loud,True,False,False,NaN,4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lK-wuiq8b1TuU7bfbQZgsg


In [17]:
temp = pd.DataFrame({'business_attributes': df_attributes.columns,
                     '%_missing': [df[column].isnull().sum() / df.shape[0]
                                   for column in df_attributes.columns]})
temp.sort_values('%_missing', inplace=True)
temp.reset_index(drop=True, inplace=True)
with pd.option_context('display.max_row', None):
    display(temp)

,business_attributes,%_missing
0,business_id,0.000000
1,BusinessAcceptsCreditCards,0.074418
2,RestaurantsTakeOut,0.088706
3,RestaurantsDelivery,0.107523
4,RestaurantsPriceRange2,0.117900
5,BusinessParking.garage,0.120881
6,BusinessParking.validated,0.120920
7,BusinessParking.street,0.120920
8,BusinessParking.valet,0.120920
9,BusinessParking.lot,0.120920


In [18]:
to_drop = temp.loc[temp['%_missing'] > 0.15, 'business_attributes'].values
df.drop(to_drop, axis=1, inplace=True)
df.drop(['attributes', 'has_attributes'], axis=1, inplace=True)
display(df.head())

,business_id,name,city,state,stars,review_count,RestaurantsTakeOut,BusinessAcceptsCreditCards,RestaurantsReservations,RestaurantsPriceRange2,RestaurantsDelivery,BusinessParking.garage,BusinessParking.street,BusinessParking.validated,BusinessParking.lot,BusinessParking.valet
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,True,True,False,1,True,NaN,NaN,NaN,NaN,NaN
1,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,Phoenix,AZ,4.0,10,True,True,False,1,False,False,False,False,True,False
2,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,Cleveland,OH,4.5,23,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
3,tLpkSwdtqqoXwU0JAGnApw,Wendy's,Cleveland,OH,3.5,7,True,True,False,1,False,False,False,False,True,False
4,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,Cleveland,OH,3.0,4,False,False,False,4,False,False,False,False,False,False


In [19]:
df.drop(['BusinessParking.garage', 'BusinessParking.validated', 'BusinessParking.street',
         'BusinessParking.valet', 'BusinessParking.lot', 'RestaurantsReservations', 
         'RestaurantsDelivery'], axis=1, inplace=True)
display(df.head())

,business_id,name,city,state,stars,review_count,RestaurantsTakeOut,BusinessAcceptsCreditCards,RestaurantsPriceRange2
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,True,True,1
1,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,Phoenix,AZ,4.0,10,True,True,1
2,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,Cleveland,OH,4.5,23,True,True,NaN
3,tLpkSwdtqqoXwU0JAGnApw,Wendy's,Cleveland,OH,3.5,7,True,True,1
4,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,Cleveland,OH,3.0,4,False,False,4


In [20]:
display(df.RestaurantsPriceRange2.unique())
display(df.RestaurantsTakeOut.unique())
display(df.BusinessAcceptsCreditCards.unique())

array(['1', nan, '4', '2', '3', 'False'], dtype=object)

array(['True', 'False', nan], dtype=object)

array(['True', 'False', nan], dtype=object)

In [21]:
df.RestaurantsTakeOut.replace(to_replace=['True', 'False'], value=[1, 0], inplace=True)
df.BusinessAcceptsCreditCards.replace(to_replace=['True', 'False'], value=[1, 0], inplace=True)
df.RestaurantsPriceRange2.fillna('NA', inplace=True)
df.RestaurantsPriceRange2 = np.where(df.RestaurantsPriceRange2 == 'False', 'NA', df.RestaurantsPriceRange2)
df.RestaurantsPriceRange2.replace(to_replace=['1', '2', '3', '4', 'NA'], value=[1, 2, 3, 4, np.nan], inplace=True)
display(df.head())

,business_id,name,city,state,stars,review_count,RestaurantsTakeOut,BusinessAcceptsCreditCards,RestaurantsPriceRange2
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,1.0,1.0,1.0
1,vjTVxnsQEZ34XjYNS-XUpA,Wetzel's Pretzels,Phoenix,AZ,4.0,10,1.0,1.0,1.0
2,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,Cleveland,OH,4.5,23,1.0,1.0,NaN
3,tLpkSwdtqqoXwU0JAGnApw,Wendy's,Cleveland,OH,3.5,7,1.0,1.0,1.0
4,lK-wuiq8b1TuU7bfbQZgsg,Hingetown,Cleveland,OH,3.0,4,0.0,0.0,4.0


In [22]:
df.to_csv('business_data.csv', index=False)